In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


지문을 읽고 주제 분류하기

다음과 같은 입력 데이터를 데이터프레임으로 저장한다. 입력 데이터는 음식 관련 및 스포츠 관련 지문으로 구성되어 있다.

In [2]:
paragraph_dict_list = [
    {'paragraph': 'Dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
    {'paragraph': 'Srvice can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
    {'paragraph': 'Portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
    {'paragraph': 'We started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
    {'paragraph': 'The biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
    {'paragraph': 'The garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
    {'paragraph': 'Our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
    {'paragraph': 'What i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
    {'paragraph': 'The drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
    {'paragraph': 'Despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    {'paragraph': 'The four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
    {'paragraph': 'The briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
    {'paragraph': 'Stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
    {'paragraph': 'When it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
    {'paragraph': 'The team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
    {'paragraph': 'The perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
    {'paragraph': 'Liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
    {'paragraph': 'Alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
    {'paragraph': 'But the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
    {'paragraph': 'Then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df

,paragraph,category
0,Dishplace is located in sunnyvale downtown the...,food
1,Srvice can be slower during busy hours but our...,food
2,Portions are huge both french toast and their ...,food
3,We started with apps going the chicken and waf...,food
4,The biscuits and gravy was too salty two peopl...,food
5,The garlic fries were a great starter (and a h...,food
6,Our meal was excellent i had the pasta ai form...,food
7,What i enjoy most about palo alto is so many r...,food
8,The drinks came out fairly quickly a good two ...,food
9,Despite the not so good burger the service was...,food


데이터 전처리

LSTM 모델이 입력 데이터를 처리할 수 있도록 수치값으로 변경한다.  
텍스트인 입력값을 수치로 변환하기 위해서 지문에 사용된 모든 단어들을 모아서 중복을 제거한 후 단어 리스트를 만든다.

In [3]:
# 중복을 제거해서 지문에 사용된 단어를 모으기 위해서 빈 set을 선언한다.
results = set()
# print(type(results)) # <class 'set'>
# print(type(df)) # <class 'pandas.core.frame.DataFrame'>
# print(type(df.paragraph)) # <class 'pandas.core.series.Series'>
# print(type(df.paragraph.str)) # <class 'pandas.core.strings.StringMethods'>
# print(type(df.paragraph.str.lower())) # <class 'pandas.core.series.Series'>
# print(type(df.paragraph.str.lower().str)) # <class 'pandas.core.strings.StringMethods'>
# print(type(df.paragraph.str.lower().str.split())) # <class 'pandas.core.series.Series'>

# 데이터프레임의 paragraph열(Series)을 모두 소문자로 변환한 후 공백을 경계로 나눠서 set에 저장한다.
# results = df.paragraph.str.lower().str.split()
# print(type(results))
df.paragraph.str.lower().str.split().apply(results.update)
print(len(results))
print(results)

537
{'saturday', 'were', '1.5x', 'iced', 'stephens', 'was', 'lunchtime', 'sweet', 'of', 'chicken', 'during', 'dinner', 'unless', 'you', 'those', 'victoria', 'no-quarter-given', 'first-round', 'pancakes', 'names', 'meal', 'reigning', 'rich', 'smash', 'ivan', 'over', 'me', 'york', 'highly', 'plus', 'sheets', 'ostapenko', 'what', 'burger', 'days', 'started', 'error', 'nn7', 'action', 'returning', 'french', 'said', 'within', 'rakitic', 'tournaments', 'breakthroughs', 'dare', 'ranking', 'performances', 'he', 'modric', 'having', 'no2', 'bookmarked', 'sisters', 'be', 'ranked', 'rankings', '2014', 'seating', 'plates', 'ago', 'full', 'table', 'avoided', 'cup', 'but', 'much', 'they', 'slow', 'reaching', 'finished', 'perseyside', 'brunch', 'goalkeeper', 'quality', 'face', 'srvice', 'eliminated', 'tea', 'for', 'without', 'off', 'no46', 'huge', 'however', 'down', 'straight', 'courteous', 'displayed', 'only', 'croatia', 'biscuits', 'would', 'semis', 'courts', 'serena', 'began—too', 'course', 'title'

set에 저장된 데이터를 아래와 같은 형태의 딕셔너리로 만든다.  
{0: 'simona', 1: 'then', 2: 'being', ..., 534: 'quality', 535: 'delivered', 536: 'me'}

In [4]:
# enumerate() 함수는 인수로 지정된 객체에 저장된 내용을 (인덱스, 데이터) 형태로 리턴한다.
# for index, word in enumerate(results):
    # print(index, word)
idx2word = dict(enumerate(results))
print(type(idx2word))
print(idx2word)

<class 'dict'>
{0: 'saturday', 1: 'were', 2: '1.5x', 3: 'iced', 4: 'stephens', 5: 'was', 6: 'lunchtime', 7: 'sweet', 8: 'of', 9: 'chicken', 10: 'during', 11: 'dinner', 12: 'unless', 13: 'you', 14: 'those', 15: 'victoria', 16: 'no-quarter-given', 17: 'first-round', 18: 'pancakes', 19: 'names', 20: 'meal', 21: 'reigning', 22: 'rich', 23: 'smash', 24: 'ivan', 25: 'over', 26: 'me', 27: 'york', 28: 'highly', 29: 'plus', 30: 'sheets', 31: 'ostapenko', 32: 'what', 33: 'burger', 34: 'days', 35: 'started', 36: 'error', 37: 'nn7', 38: 'action', 39: 'returning', 40: 'french', 41: 'said', 42: 'within', 43: 'rakitic', 44: 'tournaments', 45: 'breakthroughs', 46: 'dare', 47: 'ranking', 48: 'performances', 49: 'he', 50: 'modric', 51: 'having', 52: 'no2', 53: 'bookmarked', 54: 'sisters', 55: 'be', 56: 'ranked', 57: 'rankings', 58: '2014', 59: 'seating', 60: 'plates', 61: 'ago', 62: 'full', 63: 'table', 64: 'avoided', 65: 'cup', 66: 'but', 67: 'much', 68: 'they', 69: 'slow', 70: 'reaching', 71: 'finishe

{key: value, ...} 형태의 딕셔너리를 아래와 같이 {value: key, ...} 형태의 딕셔너리로 변환한다.  
{'simona': 0, 'then': 1, 'being': 2, ..., 'quality': 534, 'delivered': 535, 'me': 536}

In [5]:
word2idx = {value.replace('(', '').replace(')', ''): key for key, value in idx2word.items()}
print(type(word2idx))
print(word2idx)

<class 'dict'>
{'saturday': 0, 'were': 1, '1.5x': 2, 'iced': 3, 'stephens': 4, 'was': 5, 'lunchtime': 6, 'sweet': 7, 'of': 8, 'chicken': 9, 'during': 10, 'dinner': 11, 'unless': 12, 'you': 13, 'those': 14, 'victoria': 15, 'no-quarter-given': 16, 'first-round': 17, 'pancakes': 18, 'names': 19, 'meal': 20, 'reigning': 21, 'rich': 22, 'smash': 23, 'ivan': 24, 'over': 25, 'me': 26, 'york': 27, 'highly': 28, 'plus': 29, 'sheets': 30, 'ostapenko': 31, 'what': 32, 'burger': 33, 'days': 34, 'started': 35, 'error': 36, 'nn7': 37, 'action': 38, 'returning': 39, 'french': 40, 'said': 41, 'within': 42, 'rakitic': 43, 'tournaments': 44, 'breakthroughs': 45, 'dare': 46, 'ranking': 47, 'performances': 48, 'he': 49, 'modric': 50, 'having': 51, 'no2': 52, 'bookmarked': 53, 'sisters': 54, 'be': 55, 'ranked': 56, 'rankings': 57, '2014': 58, 'seating': 59, 'plates': 60, 'ago': 61, 'full': 62, 'table': 63, 'avoided': 64, 'cup': 65, 'but': 66, 'much': 67, 'they': 68, 'slow': 69, 'reaching': 70, 'finished': 

In [6]:
print(word2idx['simona'], word2idx['currently'], word2idx['halep'], word2idx['quality'])

377 443 178 75


word2idx 딕셔너리를 활용해서 모든 지문을 구성하는 단어 목록을 수치로 변환한 열을 데이터프레임에 추가한다.

In [7]:
# 지문을 구성하는 모든 단어를 수치로 변환하는 함수를 만든다.
def encode_paragraph(paragraph):
    words = paragraph.split()
    # print(words)
    encoded = []
    for word in words:
        encoded.append([word2idx[word.lower().replace('(', '').replace(')', '')]])
    # print(encoded)
    return encoded

In [8]:
# 지문이 수치로 변환된 결과를 데이터프레임에 추가한다.
df['enc_paragraph'] = df.paragraph.apply(encode_paragraph)
df

,paragraph,category,enc_paragraph
0,Dishplace is located in sunnyvale downtown the...,food,"[[403], [123], [206], [436], [257], [400], [19..."
1,Srvice can be slower during busy hours but our...,food,"[[77], [326], [55], [276], [10], [164], [453],..."
2,Portions are huge both french toast and their ...,food,"[[221], [131], [84], [435], [40], [163], [503]..."
3,We started with apps going the chicken and waf...,food,"[[376], [35], [303], [430], [420], [193], [9],..."
4,The biscuits and gravy was too salty two peopl...,food,"[[193], [92], [503], [396], [5], [531], [269],..."
5,The garlic fries were a great starter (and a h...,food,"[[193], [484], [108], [1], [332], [133], [306]..."
6,Our meal was excellent i had the pasta ai form...,food,"[[500], [20], [5], [342], [128], [141], [193],..."
7,What i enjoy most about palo alto is so many r...,food,"[[32], [128], [417], [487], [416], [168], [119..."
8,The drinks came out fairly quickly a good two ...,food,"[[193], [534], [470], [183], [518], [105], [33..."
9,Despite the not so good burger the service was...,food,"[[192], [193], [423], [157], [504], [33], [193..."


분류 항목(food[1, 0], sports[0, 1])을 원-핫 인코딩으로 수치로 변환한 열을 데이터프레임에 추가한다.

In [9]:
# 분류 항목을 원-핫 인코딩 수치로 변환하는 함수를 만든다.
def encode_category(category):
    if category == 'food':
        return [1, 0]
    else:
        return [0, 1]

In [10]:
# 분류 항목이 수치로 변환된 결과를 데이터프레임에 추가한다.
df['enc_category'] = df.paragraph.apply(encode_category)
df

,paragraph,category,enc_paragraph,enc_category
0,Dishplace is located in sunnyvale downtown the...,food,"[[403], [123], [206], [436], [257], [400], [19...","[0, 1]"
1,Srvice can be slower during busy hours but our...,food,"[[77], [326], [55], [276], [10], [164], [453],...","[0, 1]"
2,Portions are huge both french toast and their ...,food,"[[221], [131], [84], [435], [40], [163], [503]...","[0, 1]"
3,We started with apps going the chicken and waf...,food,"[[376], [35], [303], [430], [420], [193], [9],...","[0, 1]"
4,The biscuits and gravy was too salty two peopl...,food,"[[193], [92], [503], [396], [5], [531], [269],...","[0, 1]"
5,The garlic fries were a great starter (and a h...,food,"[[193], [484], [108], [1], [332], [133], [306]...","[0, 1]"
6,Our meal was excellent i had the pasta ai form...,food,"[[500], [20], [5], [342], [128], [141], [193],...","[0, 1]"
7,What i enjoy most about palo alto is so many r...,food,"[[32], [128], [417], [487], [416], [168], [119...","[0, 1]"
8,The drinks came out fairly quickly a good two ...,food,"[[193], [534], [470], [183], [518], [105], [33...","[0, 1]"
9,Despite the not so good burger the service was...,food,"[[192], [193], [423], [157], [504], [33], [193...","[0, 1]"


Dynamic RNN은 입력값의 다양한 길이를 고려하여 결과값을 출력하므로 Dynamic RNN이 각 입력값의 실제 길이(단어 개수)를 알 수 있다록 각 지문별 단어의 개수를 계산해서 계산된 값을 데이터프레임에 추가한다.

In [11]:
# 지문을 구성하는 단어의 개수를 세는 함수를 만든다.
def word_cnt(paragraph):
    return len(paragraph.split())

In [12]:
# 지문을 구성하는 단어의 개수를 데이터프레임에 추가한다.
df['seq_length'] = df.paragraph.apply(word_cnt)
df

,paragraph,category,enc_paragraph,enc_category,seq_length
0,Dishplace is located in sunnyvale downtown the...,food,"[[403], [123], [206], [436], [257], [400], [19...","[0, 1]",53
1,Srvice can be slower during busy hours but our...,food,"[[77], [326], [55], [276], [10], [164], [453],...","[0, 1]",19
2,Portions are huge both french toast and their ...,food,"[[221], [131], [84], [435], [40], [163], [503]...","[0, 1]",42
3,We started with apps going the chicken and waf...,food,"[[376], [35], [303], [430], [420], [193], [9],...","[0, 1]",43
4,The biscuits and gravy was too salty two peopl...,food,"[[193], [92], [503], [396], [5], [531], [269],...","[0, 1]",82
5,The garlic fries were a great starter (and a h...,food,"[[193], [484], [108], [1], [332], [133], [306]...","[0, 1]",24
6,Our meal was excellent i had the pasta ai form...,food,"[[500], [20], [5], [342], [128], [141], [193],...","[0, 1]",50
7,What i enjoy most about palo alto is so many r...,food,"[[32], [128], [417], [487], [416], [168], [119...","[0, 1]",43
8,The drinks came out fairly quickly a good two ...,food,"[[193], [534], [470], [183], [518], [105], [33...","[0, 1]",49
9,Despite the not so good burger the service was...,food,"[[192], [193], [423], [157], [504], [33], [193...","[0, 1]",82


텐서플로우의 RNN은 항상 같은 길이의 시퀀스를 받아야 한다.  
길이가 작은 입력 시퀀스는 패딩을 추가적으로 넣어 모든 시퀀스의 길이를 동일하게 설정한다.  
패딩이 RNN 계산에 영향을 끼치지 않도록 Dynamic RNN은 패딩 이전의 입력 시퀀스의 실제 길이를 파라미터로 받아 상태값 계산시 패딩을 제외시킨다.

In [13]:
# 최고로 긴 지문의 단어 수를 구한 후 모든 지문에 패딩을 집어넣어 최고로 긴 지문과 동일한 길이를 갖도록 만들기 위해
# 최고로 긴 지문의 단어수를 계산한다.
max_word_cnt = 0
# for row in df.paragraph:
    # if len(row.split()) > max_word_cnt:
        # max_word_cnt = len(row.split())
# for seq in df.seq_length:
    # if seq > max_word_cnt:
        # max_word_cnt = seq
max_word_cnt = df.seq_length.max()
print(max_word_cnt)

91


In [14]:
# 모든 지문에 패딩을 집어넣어 최고로 긴 문장과 동일한 길이를 갖게하는 함수를 만든다.
def sequence_padding(enc_paragraph):
    seq_length = len(enc_paragraph)
    for i in range(seq_length, max_word_cnt):
        enc_paragraph.append([-1])
    return enc_paragraph

In [15]:
# 모든 지문에 패딩이 적용된 결과를 데이터프레임에 적용한다.
df['enc_paragraph'] = df.enc_paragraph.apply(sequence_padding)
print(len(df.enc_paragraph[1]))
print(df.enc_paragraph[1])
print(len(df.enc_paragraph[12]))
print(df.enc_paragraph[12])

91
[[77], [326], [55], [276], [10], [164], [453], [66], [500], [361], [5], [88], [503], [385], [235], [275], [133], [418], [189], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]
91
[[4], [317], [517], [217], [277], [366], [501], [436], [515], [246], [525], [159], [517], [245], [368], [373], [414], [226], [56], [260], [369], [536], [167], [242], [205], [32], [332], [489], [301], [249], [536], [123], [319], [193], [383], [99], [231], [312], [213], [368], [122], [153], [70], [193], [241], [339], [193], [40], [201], [277], [372], [357], [447], [121], [95], [536], [204], [525], [193], [241], [436], [116], [90], [2

모델에서 사용할 입력값, 실제값, Dynamic RNN의 sequence_length 파라미터를 만든다.

In [16]:
# 넘파이 array() 메소드는 인수로 파이썬의 리스트나 튜플 형태의 데이터를 받아서 넘파이 배열로 변환하는 기능을 하기 때문에
# 데이터프레임의 시리즈를 바로 넣어주면 안되고 tolist() 메소드를 실행해서 시리즈를 리스트 형태로 변환한 후 넣어준다.
enc_paragraph = np.array(df.enc_paragraph.tolist()) # 입력값
# print(type(df.enc_paragraph)) # <class 'pandas.core.series.Series'>
# print(type(df.enc_paragraph.tolist())) # <class 'list'>
# print(type(enc_paragraph)) # <class 'numpy.ndarray'>
# print(enc_paragraph)
enc_category = np.array(df.enc_category.tolist()) # 실제값
# print(enc_category)
sequence_length = np.array(df.seq_length.tolist()) # Dynamic RNN의 sequence_length 파라미터값
# print(sequence_length)

train_X = enc_paragraph
train_Y = enc_category
print(train_X.shape, train_Y.shape)

(20, 91, 1) (20, 2)


텐서플로우로 지문을 읽고 주제를 분류하는 모델을 만든다.

<img src="./images/LSTM3.png" align="left" width="900"/>

문맥 벡터(contextualized vector) 생성 단계  
인덱스를 임베딩으로 변환한다. 임베딩은 학습 과정을 통해 단어 유사도를 포함하게 되어 문맥 벡터를 생성하는데 도움을 준다.  
인간의 언어(자연어)는 수치화되어있지 않은 데이터이기 때문에 머신러닝, 딥러닝 기법을 바로 사용할 수 없다. 그래서 자연어 처리에서 특징을 추출해서 수치화를 해줘야 하는데 이때 사용하는 것이 '언어의 벡터화'이다. 이런 벡터화 과정을 word embedding 이라고 한다.  
LSTM에 임베딩된 시퀀스를 입력해서 최종 상태값을 출력한다. => 최종 상태값이 문맥 벡터이다.

주제(food, sports) 분류 단계  
문맥 벡터를 덴즈 레이어에 입력하고 출력값을 노드가 2개인 덴즈 레이어에 입력한 후 노드가 2개인 덴즈 레이어의 출력값을 소프트 맥스에 입력시켜 food, sports에 대한 예측값을 계산한다.

모델을 구현한다.

In [17]:
# 초기화
tf.set_random_seed(35)
learning_rate = 0.001 # 학습율
n_epoch = 300 # 학습 횟수

# 입력값과 출력값을 저장할 placeholder를 만든다.
# 입력값은 문장을 구성하는 단어들의 인덱스이며 그 길이는 지문을 구성하는 단어의 최대 개수(max_word_cnt)이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, max_word_cnt, 1]) # 입력값을 기억할 placeholder, (20, 91, 1)
Y = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # 실제값을 기억할 placeholder, (20, 2)

# 워드 임베딩 레이어, 임베딩 레이어는 입력값(인덱스)을 입력받아 5차원 벡터의 임베딩을 출력한다.
# layers.dense() 메소드는 densely-connect layer 즉 '완전 연결 계층'을 만들어 준다.
embedding = tf.layers.dense(X, 5)
# LSTM 셀은 64차원 벡터의 생성값을 출력한다.
cell = tf.nn.rnn_cell.LSTMCell(num_units=64)

# 출력값과 상태값을 저장한다. 상태값이 문맥 벡터로 사용되며 dense layer의 입력값이 된다.
outputs, state = tf.nn.dynamic_rnn(cell, embedding ,dtype=tf.float32, sequence_length=sequence_length)

# 주제 분류는 두 개의 덴즈 레이어를 사용한다.
# 첫 번째 덴즈 레이어는 32개 노드를 가지고 있고 두 번째 덴즈 레이어는 2개의 노드를 가지고 있으며 이 2개의 노드가 
# 소프트 맥스의 입력으로 들어간다.
dense_layer = tf.layers.dense(state.h, 32)
# logit은 food, sports를 원-핫 인코딩으로 구분하기 위해 2차원 벡터로 구성한다.
logit = tf.layers.dense(dense_layer, 2)

# 손실 함수 - 크로스 엔트로피
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y)
loss = tf.reduce_mean(cross_entropy)

# Adam 옵티마이저
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


모델 요약

In [18]:
# 문맥 벡터 생성
# 입력값은 단어들의 인덱스이며 그 길이는 항상 91이다.
print(X) # Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
# 임베딩 레이어는 인덱스를 받아 5차원 벡터의 임베딩을 출력한다.
print(embedding) # Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
# LSTM 셀은 64차원의 상태값을 출력한다.
print(state.h) # Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)

# 문맥 벡터를 사용해 지문의 주제 분류하기, 주제 분류는 2개의 덴즈 레이어를 사용한다.
# 첫 번째 덴즈 레이어는 32개의 노드를 가지고 있다.
print(dense_layer) # Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
# 두 번째 덴즈 레이어는 2개의 노드를 가지고 있으며, 이 2개의 노드는 소프트 맥스의 입력값으로 들어간다.
print(logit) # Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)

Tensor("Placeholder:0", shape=(?, 91, 1), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, 91, 5), dtype=float32)
Tensor("rnn/while/Exit_4:0", shape=(?, 64), dtype=float32)
Tensor("dense_1/BiasAdd:0", shape=(?, 32), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 2), dtype=float32)


학습 시킨다.

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epoch + 1):
        # sess.run(optimizer, feed_dict={X: train_X, Y: train_Y})
        # train_loss = sess.run(loss, feed_dict={X: train_X, Y: train_Y})
        _, train_loss = sess.run([optimizer, loss], feed_dict={X: train_X, Y: train_Y})
        
        if epoch % 50 == 0:
            predict = tf.nn.softmax(logit) # 예측
            # 정확도 계산
            correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(Y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
            cur_acc = accuracy.eval(feed_dict={X: train_X, Y: train_Y})
            print('epoch: {:3d}, loss: {:6.4f}, acc: {:6.4f}'.format(epoch, train_loss, cur_acc))

epoch:   0, loss: 0.4563, acc: 1.0000
epoch:  50, loss: 0.0000, acc: 1.0000
epoch: 100, loss: 0.0000, acc: 1.0000
epoch: 150, loss: 0.0000, acc: 1.0000
epoch: 200, loss: 0.0000, acc: 1.0000
epoch: 250, loss: 0.0000, acc: 1.0000
epoch: 300, loss: 0.0000, acc: 1.0000
